<a href="https://colab.research.google.com/github/smitjiwani/reinforcement-learning/blob/main/rl4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install dependencies
!pip uninstall numpy -y
!pip install numpy==1.24.3
!pip install gym[classic_control]==0.26.2

Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Successfully uninstalled numpy-1.24.3
  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.3 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
blosc2 3.3.0 requires numpy>=1.26, but you have numpy 1.24.3 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompa

  Using cached gym-0.26.2.tar.gz (721 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pygame-2.1.0.tar.gz (5.8 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
#import required libraries
import gym
import numpy as np
env = gym.make('FrozenLake-v1')

/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


**Evaluate the given policy for the given environment using iterative policy evaluation.**


> Add blockquote



In [ ]:
def policy_evaluation(policy, environment, discount_factor=1.0, theta=1e-9, max_iterations=1e9):
    V = np.zeros(environment.observation_space.n)

    for i in range(int(max_iterations)):
        delta = 0
        for state in range(environment.observation_space.n):
            v = 0
            for action, action_probability in enumerate(policy[state]):
                for state_probability, next_state, reward, terminated in environment.P[state][action]:
                    v += action_probability * state_probability * (reward + discount_factor * V[next_state])

            delta = max(delta, np.abs(V[state] - v))
            V[state] = v

        evaluation_iterations += 1
        if delta < theta:
            print(f'Policy evaluated in {evaluation_iterations} iterations.')
            return V

    print(f'Maximum iterations reached. Policy evaluation terminated after {evaluation_iterations} iterations.')
    return V

In [ ]:

def one_step_lookahead(environment, state, V, discount_factor):
    action_values = np.zeros(environment.action_space.n)
    for action in range(environment.action_space.n):
        for probability, next_state, reward, terminated in environment.P[state][action]:
            action_values[action] += probability * (reward + discount_factor * V[next_state])
    return action_values

In [ ]:
def policy_iteration(environment, discount_factor=1.0, max_iterations=1e9):
    policy = np.ones([environment.observation_space.n, environment.action_space.n]) / environment.action_space.n
    evaluated_policies = 1
    for i in range(int(max_iterations)):
        stable_policy = True
        V = policy_evaluation(policy, environment, discount_factor=discount_factor)
        for state in range(environment.observation_space.n):
            current_action = np.argmax(policy[state])
            action_value = one_step_lookahead(environment, state, V, discount_factor)
            best_action = np.argmax(action_value)
            if current_action != best_action:
                stable_policy = True
                policy[state] = np.eye(environment.action_space.n)[best_action]
        evaluated_policies += 1
        if stable_policy:
            print(f'Evaluated {evaluated_policies} policies.')
            return policy, V

In [ ]:

def value_iteration(environment, discount_factor=1.0, theta=1e-9, max_iterations=1e9):
    V = np.zeros(environment.observation_space.n)
    for i in range(int(max_iterations)):
        delta = 0
        for state in range(environment.action_space.n):
            action_value = one_step_lookahead(environment, state, V, discount_factor)
            best_action_value = np.max(action_value)
            delta = max(delta, np.abs(V[state] - best_action_value))
            V[state] = best_action_value
        if delta < theta:
            print(f'Value-iteration converged at iteration#{i}.')
            break

    policy = np.zeros([environment.observation_space.n, environment.action_space.n])
    for state in range(environment.observation_space.n):
        action_value = one_step_lookahead(environment, state, V, discount_factor)
        best_action = np.argmax(action_value)
        policy[state, best_action] = 1.0
    return policy, V

In [ ]:

def play_episodes(environment, n_episodes, policy):
    wins = 0
    total_reward = 0
    for episode in range(n_episodes):
        terminated = False
        state = environment.reset()
        while not terminated:
            action = np.argmax(policy[state])
            next_state, reward, terminated, info = environment.step(action)
            total_reward += reward
            state = next_state
            if terminated and reward == 1.0:
                    wins += 1
    average_reward = total_reward / n_episodes
    return wins, total_reward, average_reward

In [ ]:
def policy_evaluation(policy, environment, discount_factor=1.0, theta=1e-9, max_iterations=1e9):
    """
    Evaluates a policy given an environment and a discount factor.

    Args:
        policy: A 2D numpy array representing the policy.
        environment: The OpenAI Gym environment.
        discount_factor: The discount factor.
        theta: The threshold for convergence.
        max_iterations: The maximum number of iterations.

    Returns:
        A numpy array representing the value function.
    """
    V = np.zeros(environment.observation_space.n)
    # Initialize evaluation_iterations to 0 before using it
    evaluation_iterations = 0

    for i in range(int(max_iterations)):
        delta = 0
        for state in range(environment.observation_space.n):
            v = 0
            for action, action_probability in enumerate(policy[state]):
                for state_probability, next_state, reward, terminated in environment.P[state][action]:
                    v += action_probability * state_probability * (reward + discount_factor * V[next_state])

            delta = max(delta, np.abs(V[state] - v))
            V[state] = v

        evaluation_iterations += 1
        if delta < theta:
            print(f'Policy evaluated in {evaluation_iterations} iterations.')
            return V

    print(f'Maximum iterations reached. Policy evaluation terminated after {evaluation_iterations} iterations.')
    return V

In [ ]:
n_episodes = 1000
policy, V = policy_iteration(env)
wins, total_reward, average_reward = play_episodes(env, n_episodes, policy)
print(f'Wins: {wins}/{n_episodes}')

/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Policy evaluated in 65 iterations.
Evaluated 2 policies.
Wins: 714/1000


In [ ]:
n_episodes = 1000

solvers = [('Policy Iteration', policy_iteration),
           ('Value Iteration', value_iteration)]

for iteration_name, iteration_func in solvers:

    environment = gym.make('FrozenLake-v1')

    policy, V = iteration_func(environment.env)
    wins, total_reward, average_reward = play_episodes(env, n_episodes, policy)
    print(f'{iteration_name} :: number of wins over {n_episodes} episodes = {wins}')
    print(f'{iteration_name} :: average reward over {n_episodes} episodes = {average_reward}\n\n')

Policy evaluated in 65 iterations.
Evaluated 2 policies.
Policy Iteration :: number of wins over 1000 episodes = 727
Policy Iteration :: average reward over 1000 episodes = 0.727


Value-iteration converged at iteration#0.
Value Iteration :: number of wins over 1000 episodes = 0
Value Iteration :: average reward over 1000 episodes = 0.0


